In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("TestSpark") \
    .master("local[*]") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
df = spark.read.json("./*.json")


In [5]:
result = df.groupBy("language").count()

In [6]:
result.show()

[Stage 1:>                                                        (0 + 11) / 11]

+----------+-----+
|  language|count|
+----------+-----+
|        C#|  336|
|     Terra|    1|
|  Makefile|   36|
|       VBA|    3|
|      Cuda|   12|
|       Arc|    1|
|      Rust|  165|
|JavaScript| 5293|
|      TSQL|    9|
|      Perl|   14|
|    Puppet|    4|
|    Erlang|    6|
|      null| 1425|
|     Jinja|   14|
|       C++|  952|
|        F#|    2|
|    Groovy|   16|
|       TeX|   43|
|     OCaml|   16|
|      MQL5|    1|
+----------+-----+
only showing top 20 rows



In [14]:
pg2_url = "jdbc:postgresql://hadoop-hive-spark-docker-master-metastore-1:5432/sparkdb"

In [15]:
result.write.format("jdbc").option("url", pg2_url).option("dbtable", "programming_lang").option("user", "postgres").option("password", "1234").option("driver", "org.postgresql.Driver").mode("append").save()

In [17]:
from pyspark.sql.functions import sum, col

In [18]:
result2 = df.filter(df['type'] == 'Organization').groupBy('username').agg(sum(col('stars').cast('int')).alias('total_stars'))

In [19]:
result2.show()

+------------------+-----------+
|          username|total_stars|
+------------------+-----------+
|              didi|       4914|
|          twosigma|       6150|
|       intel-spark|         91|
|     Azure-Samples|       1398|
|            seznam|         81|
|          Qihoo360|      18582|
|          snowlift|         26|
|    adobe-research|        367|
|music-of-the-ainur|         22|
|           Alluxio|      16520|
|            cdapio|         81|
|    USCDataScience|        390|
|            cerner|        128|
|    HadoopGenomics|         70|
|     dk-stationery|         11|
|          uosdmlab|         53|
|             arkig|          5|
|         OpenMined|      16100|
|      Salmon-Brain|         10|
|       oap-project|          7|
+------------------+-----------+
only showing top 20 rows



In [20]:
result2.write.format("jdbc").option("url", pg2_url).option("dbtable", "organizations_stars").option("user", "postgres").option("password", "1234").option("driver", "org.postgresql.Driver").mode("append").save()

In [21]:
from pyspark.sql.functions import input_file_name, regexp_extract

In [22]:
df2=spark.read.json("./*.json").withColumn("search_term",input_file_name())

In [25]:
df2 = df2.withColumn("search_term", regexp_extract(df2["search_term"], r"([^/]+)\.json$", 1))

In [28]:
df2.show()

+---------------+-------------------+--------------------+-----+--------------------+---------+----------------+-----------+--------------------+-----+-----------+--------------------+------------+-------------------+-----------+
|_corrupt_record|            created|         description|forks|           full_name|       id|        language|open_issues|           repo_name|stars|subscribers|              topics|        type|           username|search_term|
+---------------+-------------------+--------------------+-----+--------------------+---------+----------------+-----------+--------------------+-----+-----------+--------------------+------------+-------------------+-----------+
|           null|2014-02-25 08:00:08|Apache Spark - A ...|25357|        apache/spark| 17165658|           Scala|        242|               spark|32296|       2080|[python, scala, r...|Organization|             apache|      Spark|
|           null|2017-08-09 19:39:59|Distributed train...| 2027|     horovod/hor

In [30]:
df2 = df2.withColumn("relevance_score" , 1.5 * col("forks") + 1.32 * col("subscribers") + 1.04 * col("stars"))

In [31]:
df2_grouped = df2.groupBy("search_term").agg(sum("relevance_score").alias("relevance_score"))

In [32]:
df2_grouped.show()

+------------+------------------+
| search_term|   relevance_score|
+------------+------------------+
|       Spark| 295286.1599999999|
|      Hadoop| 584015.9000000012|
|      Kotlin|1936763.9000000004|
|    React-JS| 870068.3600000012|
|  Tensorflow| 3918987.260000002|
|      Docker| 4403817.560000001|
|         Cpp| 4377811.440000002|
|    ethereum| 1157287.259999999|
|       Flask| 884822.1200000001|
|     Flutter|              null|
|       Scala|1287463.6800000004|
|     PyTorch| 3357329.079999997|
|React-Native|3097924.4600000037|
|  Typescript| 4909697.760000003|
|  Kubernetes|3540183.0199999996|
|      Scikit| 897061.2400000009|
|           R|     1.187633824E7|
|      NodeJS| 4331014.679999995|
|     Threejs|360260.49999999965|
|      Django| 1488257.160000001|
+------------+------------------+
only showing top 20 rows



In [33]:
result.write.format("jdbc").option("url", pg2_url).option("dbtable", "search_terms_relevance").option("user", "postgres").option("password", "1234").option("driver", "org.postgresql.Driver").mode("append").save()